![Picture title](image-20211026-161155.png)

In [ ]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
bb2014=pd.read_csv('sample_data/2014-baby-names-illinois.csv')
bb2015=pd.read_csv('sample_data/2015-baby-names-illinois.csv')
chpt=pd.read_csv('sample_data/chipotle.csv')
blb=pd.read_csv('sample_data/billboard.csv', encoding="mac_latin2")
pew=pd.read_csv('sample_data/pew.csv')
tb=pd.read_csv('sample_data/tb.csv')
tbnot=pd.read_csv('sample_data/TB_notifications_2019-10-15.csv')
weather=pd.read_csv('sample_data/weather.csv')
weatd=pd.read_csv('sample_data/weather_tidy.csv')

In [ ]:
pew=pew.rename(columns={'<$10k':'$0-10k'})

## Data set Pew forum

Problem: The columns headers are composed of the possible income values so we need organize these values to relationship religion to income and do a data cleaning. This is a messy data set; a tidy version of this dataset is one in which the income values would not be columns headers but rather values in an&nbsp;income&nbsp;column. In order to tidy this dataset, we need to&nbsp;melt&nbsp;it

In [ ]:
pewtidy = pd.melt(pew, ["religion"], var_name="income", value_name="frequency")
pewtidy = pewtidy.sort_values(by=["frequency"])
pewtidy.head(5)

,religion,income,frequency
6,Hindu,$0-10k,1
34,Other World Religions,$10-20k,2
88,Other World Religions,$40-50k,2
52,Other World Religions,$20-30k,3
124,Other World Religions,$75-100k,3


In [ ]:
pewtidy.corr(method= 'spearman')
#'pearson', 'spearman', 'kendall', or a callable

,frequency
frequency,1.0


In [ ]:
pewtidy.info

<bound method DataFrame.info of                   religion              income  frequency
6                    Hindu              $0-10k          1
34   Other World Religions             $10-20k          2
88   Other World Religions             $40-50k          2
52   Other World Religions             $20-30k          3
124  Other World Religions            $75-100k          3
..                     ...                 ...        ...
93                Catholic             $50-75k       1116
172          Mainline Prot  Don't know/refused       1328
95        Evangelical Prot             $50-75k       1486
165               Catholic  Don't know/refused       1489
167       Evangelical Prot  Don't know/refused       1529

[180 rows x 3 columns]>

In [ ]:
pewtidy['income']=pewtidy['income'].astype('category').cat.codes
pewtidy['religion']=pewtidy['religion'].astype('category').cat.codes
#pewtidy['Atheist']=pewtidy['Atheist'].astype('category').cat.codes
#pewtidy['Buddhist']=pewtidy['Buddhist'].astype('category').cat.codes
pewtidy.corr()
#Agnostic Atheist Buddhist Catholic Don't know/refused, Evangelical Prot, Hindu  

,religion,income,frequency
religion,1.000000,0.00000,-0.149554
income,0.000000,1.00000,0.135840
frequency,-0.149554,0.13584,1.000000


In [ ]:
pewtidy.corr

<bound method DataFrame.corr of      religion  income  frequency
6           6       0          1
34         16       1          2
88         16       5          2
52         16       3          3
124        16       7          3
..        ...     ...        ...
93          3       6       1116
172        10       9       1328
95          5       6       1486
165         3       9       1489
167         5       9       1529

[180 rows x 3 columns]>

In [ ]:
#Finding the special characters in the data frame 
pewtidy.isin(['$']).sum(axis=0)

religion     0
income       0
frequency    0
dtype: int64

In [ ]:
!pip install pandas-profiling==2.7.1
from pandas_profiling import ProfileReport
profile = ProfileReport(pewtidy)
#profile.to_widgets() --> to display the data in widgets

     |████████████████████████████████| 252 kB 5.0 MB/s 
     |████████████████████████████████| 4.7 MB 50.3 MB/s 
     |████████████████████████████████| 675 kB 52.5 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 812 kB 54.0 MB/s 
     |████████████████████████████████| 28.5 MB 49 kB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=6e11c49f7fa426f1ba14224ac58017413c266a7002877a1394244ea21f902040
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=7e015dc62149266504a38ac6541d7d41ea4b39bdc84aea07f71b53fc15b49263
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built htmlmin imagehash
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling

In [ ]:
profile

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Billboard Data set
The problem of this data set is that every week and rank are in a column. So we need to split them and reallocate them in separate columns. Also, the data set had som specific characters, so we need to lod it with an specific encoder.

In [ ]:
blb

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,x4th.week,x5th.week,x6th.week,x7th.week,x8th.week,x9th.week,x10th.week,x11th.week,x12th.week,x13th.week,x14th.week,x15th.week,x16th.week,x17th.week,x18th.week,x19th.week,x20th.week,x21st.week,x22nd.week,x23rd.week,x24th.week,x25th.week,x26th.week,x27th.week,x28th.week,x29th.week,x30th.week,x31st.week,x32nd.week,x33rd.week,...,x37th.week,x38th.week,x39th.week,x40th.week,x41st.week,x42nd.week,x43rd.week,x44th.week,x45th.week,x46th.week,x47th.week,x48th.week,x49th.week,x50th.week,x51st.week,x52nd.week,x53rd.week,x54th.week,x55th.week,x56th.week,x57th.week,x58th.week,x59th.week,x60th.week,x61st.week,x62nd.week,x63rd.week,x64th.week,x65th.week,x66th.week,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,33.0,23.0,15.0,7.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,7.0,10.0,12.0,15.0,22.0,29.0,31.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,5.0,2.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0,15.0,19.0,21.0,26.0,36.0,48.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,31.0,20.0,13.0,7.0,6.0,4.0,4.0,4.0,6.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,8.0,8.0,12.0,14.0,17.0,21.0,24.0,30.0,34.0,37.0,46.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,14.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,11.0,16.0,20.0,25.0,27.0,27.0,29.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,29.0,23.0,18.0,11.0,9.0,9.0,11.0,1.0,1.0,1.0,1.0,4.0,8.0,12.0,22.0,23.0,43.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000-08-05,2000-08-05,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,2000,"Smith, Will",Freakin' It,3:58,Rap,2000-02-12,2000-02-12,99,99.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,99,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
id_vars = ["year","artist.inverted", "track","time","genre","date.entered","date.peaked"]
blb = pd.melt(frame = blb,id_vars=id_vars, var_name = "week", value_name = "rank")

blb["week"] = blb['week'].str.extract('(\d+)', expand=False).astype('int', errors='ignore')
blb["rank"] = blb["rank"].astype('int', errors='ignore')

blb = blb.dropna()

blb['date'] = pd.to_datetime(blb['date.entered']) + pd.to_timedelta(blb['week'], unit='w') - pd.DateOffset(weeks=1)

blb = blb[["year","artist.inverted","track","time","genre","week","rank","date"]]
blb = blb.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

blbtidy = blb

In [ ]:
blbtidy

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87.0,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82.0,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72.0,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77.0,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87.0,2000-03-25
...,...,...,...,...,...,...,...,...
10793,2000,matchbox twenty,Bent,4:12,Rock,35,33.0,2000-12-23
11110,2000,matchbox twenty,Bent,4:12,Rock,36,37.0,2000-12-30
11427,2000,matchbox twenty,Bent,4:12,Rock,37,38.0,2001-01-06
11744,2000,matchbox twenty,Bent,4:12,Rock,38,38.0,2001-01-13


# Tuberculosis data set 
Patients sex and age range are ordered as single columns. So we need to split it and create gender and age range table. Also iso2 is actually countries. So we need to rename it.

In [ ]:
tb

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,new_sp_m65,new_sp_mu,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,ZW,2004,14581.0,NaN,NaN,187.0,833.0,2908.0,2298.0,1056.0,366.0,198.0,NaN,NaN,NaN,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,NaN
5765,ZW,2005,13155.0,NaN,NaN,210.0,837.0,2264.0,1855.0,762.0,295.0,656.0,NaN,NaN,NaN,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,NaN
5766,ZW,2006,12718.0,NaN,NaN,215.0,736.0,2391.0,1939.0,896.0,348.0,199.0,NaN,NaN,NaN,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,NaN
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,292.0,153.0,NaN,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,NaN


In [ ]:
tb = pd.melt(tb, id_vars=["iso2","year"], value_name="cases", var_name="sex_and_age")

temp = tb["sex_and_age"].str.extract("(\D)(\d+)(\d{2})")    

temp.columns = ["sex", "age_lower", "age_upper"]

temp["age"] = temp["age_lower"] + "-" + temp["age_upper"]

tb_tidy = pd.concat([tb, temp], axis=1)

tb_tidy = tb_tidy.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
tb_tidy = tb_tidy.dropna()

tb_tidy = tb_tidy.rename(columns={'iso2':'country'})

tb_tidy

,country,year,cases,sex,age
11553,AD,2005,0.0,m,5-14
11554,AD,2006,0.0,m,5-14
11556,AD,2008,0.0,m,5-14
11580,AE,2006,0.0,m,5-14
11581,AE,2007,2.0,m,5-14
...,...,...,...,...,...
109606,ZW,2004,214.0,f,55-64
109607,ZW,2005,193.0,f,55-64
109608,ZW,2006,230.0,f,55-64
109609,ZW,2007,213.0,f,55-64


### Problem: Weather Data Set

Variables are stored in tmin,&nbsp;tmax and columns (days), which is not effective and really difficult to read.

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       22 non-null     object 
 1   year     22 non-null     float64
 2   month    22 non-null     float64
 3   element  22 non-null     object 
 4   d1       2 non-null      float64
 5   d2       4 non-null      float64
 6   d3       4 non-null      float64
 7   d4       2 non-null      float64
 8   d5       8 non-null      float64
 9   d6       2 non-null      float64
 10  d7       2 non-null      float64
 11  d8       2 non-null      float64
 12  d9       0 non-null      float64
 13  d10      2 non-null      float64
 14  d11      2 non-null      float64
 15  d12      0 non-null      float64
 16  d13      2 non-null      float64
 17  d14      4 non-null      float64
 18  d15      2 non-null      float64
 19  d16      2 non-null      float64
 20  d17      2 non-null      float64
 21  d18      0 non-nul

In [ ]:
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX000017004,2010.0,1.0,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.0,NaN
1,MX000017004,2010.0,1.0,TMIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN
2,MX000017004,2010.0,2.0,TMAX,NaN,273.0,241.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,297.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX000017004,2010.0,2.0,TMIN,NaN,144.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX000017004,2010.0,3.0,TMAX,NaN,NaN,NaN,NaN,321.0,NaN,NaN,NaN,NaN,345.0,NaN,NaN,NaN,NaN,NaN,311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
weather_melt=pd.melt(weather,
id_vars=['id','year', 'month', 'element'], 
var_name='day', 
value_name='temp')

In [ ]:
weather_melt.head()

,id,year,month,element,day,temp
0,MX000017004,2010.0,1.0,TMAX,d1,NaN
1,MX000017004,2010.0,1.0,TMIN,d1,NaN
2,MX000017004,2010.0,2.0,TMAX,d1,NaN
3,MX000017004,2010.0,2.0,TMIN,d1,NaN
4,MX000017004,2010.0,3.0,TMAX,d1,NaN


In [ ]:
weathertd=weather_melt.pivot_table(
    index=['id', 'year', 'month', 'day'],
    columns='element',
    values='temp')

In [ ]:
print(weathertd[0:60])

element                        TMAX   TMIN
id          year   month day              
MX000017004 2010.0 1.0   d30  278.0  145.0
                   2.0   d11  297.0  134.0
                         d2   273.0  144.0
                         d23  299.0  107.0
                         d3   241.0  144.0
                   3.0   d10  345.0  168.0
                         d16  311.0  176.0
                         d5   321.0  142.0
                   4.0   d27  363.0  167.0
                   5.0   d27  332.0  182.0
                   6.0   d17  280.0  175.0
                         d29  301.0  180.0
                   7.0   d3   286.0  175.0
                         d14  299.0  165.0
                   8.0   d23  264.0  150.0
                         d5   296.0  158.0
                         d29  280.0  153.0
                         d13  298.0  165.0
                         d25  297.0  156.0
                         d31  254.0  154.0
                         d8   290.0  173.0
           

In [ ]:
weather_tidy=weathertd.reset_index()